In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import random
from itertools import permutations
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum
from typing import Dict, List, Tuple
from pulp import GLPK_CMD

def generate_dummy_data(n_pois: int, n_days: int) -> tuple:
    """
    Generate dummy data for the itinerary optimization problem.

    Parameters:
    - n_pois: The number of points of interest (POIs).
    - n_days: The number of days.

    Returns:
    - pois: A list of POI names.
    - days: A list of day identifiers.
    - travel_time: A dictionary with travel times between each pair of POIs.
    """
    pois = [f'poi_{i}' for i in range(n_pois)]
    days = [f'D{day}' for day in range(1, n_days + 1)]
    travel_time = {(poi1, poi2): random.randint(10, 60) for poi1, poi2 in permutations(pois, 2)}
    return pois, days, travel_time



def extract_itinerary(nodes):
    # Step 1: Group nodes by day
    day_edges = {}
    for node in nodes:
        poi1, poi2, day = node
        if day not in day_edges:
            day_edges[day] = []
        day_edges[day].append((poi1, poi2))
    
    itineraries = {}
    # Step 2: Construct the graph for each day and find the path
    for day, edges in day_edges.items():
        graph = {}
        in_degree = {}  # Track incoming edges
        for poi1, poi2 in edges:
            graph[poi1] = poi2
            # Update in_degree counts
            in_degree[poi2] = in_degree.get(poi2, 0) + 1
            if poi1 not in in_degree:
                in_degree[poi1] = 0
        
        # Step 3: Find the starting POI (no incoming edges) and construct the itinerary
        start_poi = None
        for poi, in_deg in in_degree.items():
            if in_deg == 0:
                start_poi = poi
                break
        
        itinerary = [start_poi]
        while start_poi in graph:
            next_poi = graph[start_poi]
            itinerary.append(next_poi)
            start_poi = next_poi
        
        itineraries[day] = itinerary
    
    return itineraries

In [55]:
# Initialize data
n_pois = 10  # Number of POIs
n_days = 6  # Number of days
pois, days, travel_time = generate_dummy_data(n_pois, n_days)


n_nonce_nodes = 2 # random.randint(2, 3)
nonce_nodes = [f"nonce_node_{str(i)}" for i in range(n_nonce_nodes)]

# Initialize the problem
problem = LpProblem("Itinerary_Optimization", LpMinimize)

# Variables
order_vars = {
    (poi1, poi2, day): LpVariable(f"order_{poi1}_{poi2}_{day}", cat="Binary")
    for poi1 in pois + nonce_nodes for poi2 in pois + nonce_nodes 
    for day in days if poi1 != poi2
}
# Objective function: Minimize the total travel time
problem += lpSum(
    travel_time[(poi_from, poi_to)] * order_vars[(poi_from, poi_to, day)]
    for poi_from, poi_to in permutations(pois, 2) for day in days
)

# Constraints

# Each POI visited exactly once across all days
for poi in pois:
    problem += lpSum(
        order_vars[(poi, other_poi, day)] + order_vars[(other_poi, poi, day)]
        for day in days for other_poi in pois + nonce_nodes if poi != other_poi
    ) == 2, f"enter_and_exit_{poi}"

# Ensure that starting nonce_nodes are visited n_days times
problem += lpSum(order_vars[(nonce_node, poi, day)] 
                 for poi in pois for day in days
                 for nonce_node in nonce_nodes) == n_days, "start_nonce"

# Ensure that ending nonce_nodes are visited n_days times
problem += lpSum(order_vars[(poi, nonce_node, day)] 
                 for poi in pois for day in days
                 for nonce_node in nonce_nodes) == n_days, "end_nonce"

for day in days:
    problem += lpSum(
        order_vars[(nonce_node, poi, day)] 
        for poi in pois for nonce_node in nonce_nodes) == 1, f"start_nonce_{day}"
    
    problem += lpSum(
        order_vars[(poi, nonce_node, day)] 
        for poi in pois for nonce_node in nonce_nodes) == 1, f"end_nonce_{day}"


# Ensure entering and exiting each POI occurs within the same day
for day in days:
    for poi in pois:
        problem += lpSum(
            order_vars[(poi, other_poi, day)]
            for other_poi in pois + nonce_nodes if poi != other_poi
        ) == lpSum(
            order_vars[(other_poi, poi, day)]
            for other_poi in pois + nonce_nodes if poi != other_poi
        ), f"same_day_visit_{poi}_{day}"




for nonce_node in nonce_nodes:
    for day in days:
        problem += lpSum(order_vars[(nonce_node, poi, day)] for poi in pois) <= 1


for nonce_node in nonce_nodes:
    for poi in pois:
        problem += lpSum(order_vars[(nonce_node, poi, day)] for day in days) <= 1

# problem
# Solve the problem
status = problem.solve(GLPK_CMD(msg=0))

# Check the status and print the results
if LpStatus[status] == 'Optimal':
    print("Solution Found:\n")
    results = []
    for var in order_vars:
        if order_vars[var].varValue == 1:
            results.append(var)
            print(f"{var} = {order_vars[var].varValue}")
else:
    print("No optimal solution found.")


Solution Found:

('poi_0', 'nonce_node_1', 'D1') = 1
('poi_1', 'poi_4', 'D6') = 1
('poi_2', 'nonce_node_1', 'D3') = 1
('poi_3', 'nonce_node_1', 'D5') = 1
('poi_4', 'poi_5', 'D6') = 1
('poi_5', 'poi_1', 'D6') = 1
('poi_6', 'nonce_node_1', 'D2') = 1
('poi_7', 'nonce_node_1', 'D4') = 1
('poi_8', 'nonce_node_1', 'D6') = 1
('poi_9', 'poi_8', 'D6') = 1
('nonce_node_0', 'poi_0', 'D1') = 1
('nonce_node_0', 'poi_2', 'D3') = 1
('nonce_node_0', 'poi_6', 'D2') = 1
('nonce_node_0', 'poi_9', 'D6') = 1
('nonce_node_1', 'poi_3', 'D5') = 1
('nonce_node_1', 'poi_7', 'D4') = 1
